# March Mania 2025 -
# Forecast the 2025 NCAA Basketball Tournaments

## Overview of submission strategy 

1. RandomForest, XGBoost, LightGBM을 기반으로 한 스태킹 앙상블 모델의 베이스라인 코드
2. 538 예측 데이터, Massey Ordinals(팀별 순위 데이터) 등 외부 평가 기관 순위 데이터를 활용해 시즌 트렌드를 반영한 피처로 활용 -> 홈/원정 경기 승률, 코치 변경 이력 등
3. 개별 모델의 예측값을 Logistic Regression 메타 모델로 결합하는 방식
4. 기본 피처에는 538 예측값, Massey Ordinals, 기타 팀 성과 지표를 포함한다고 가정할 것임

# 1. Library import

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import log_loss
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
import lightgbm as lgb

## <토너먼트 경기 결과 예측 피처>
- 시드 차이
- 538의 승률 예측
- Massey Ordinals
- 공격 및 수비 효율성
- 최근 경기 승률
- 토너먼트 경험 및 과거 성적
- 컨퍼런스 강도(경쟁력) 및 대회 성적
- 팀의 경기 스타일 및 템포
- 개별 선수 및 코칭 요소(부상 여부, 복귀 상황, 코치 전략, 코치 운영 능력 등)
- 조정된 효율성 지표(상대의 수비력을 고려한 공격효율성, 상대의 공격력을 고려한 수비 효율성)
- 추가 외부 지표

# 2. Feature Engineering

## #1. 시드 차이(SeedDiff) 계산(submission_with_seed_diff)
 - 제출 파일과 남녀의 시드 파일을 불러와 게임 정보와 각 팀의 시드 값을 추출한 후 병합하여 시드 차이 계산

In [ ]:
# 1. 랜덤 시드 설정 (재현성 확보)
np.random.seed(42)

# 2. 시드 데이터 불러오기 (남녀 합침)
w_seed = pd.read_csv('/kaggle/input/march-machine-learning-mania-2025/WNCAATourneySeeds.csv')
m_seed = pd.read_csv('/kaggle/input/march-machine-learning-mania-2025/MNCAATourneySeeds.csv')
seed_df = pd.concat([m_seed, w_seed], ignore_index=True).fillna(0.05)  # 결측치 기본값 처리

# 3. 시드 문자열에서 숫자 값만 추출하는 함수
def extract_seed_value(seed_str):
    try:
        return int(''.join(filter(str.isdigit, seed_str)))  # 숫자만 추출
    except (ValueError, TypeError):
        return 16  # 기본값

seed_df['SeedValue'] = seed_df['Seed'].apply(extract_seed_value)

# 4. 제출 데이터 불러오기
submission_df = pd.read_csv('/kaggle/input/march-machine-learning-mania-2025/SampleSubmissionStage1.csv')

# 5. ID에서 시즌과 두 팀 ID 추출하는 함수
def extract_game_info(id_str):
    parts = id_str.split('_')
    return int(parts[0]), int(parts[1]), int(parts[2])

# 6. 시즌, 팀 ID 분리해서 컬럼 추가
submission_df[['Season', 'TeamID1', 'TeamID2']] = submission_df['ID'].apply(extract_game_info).tolist()

# 7. TeamID1에 대한 시드 정보 병합
submission_df = submission_df.merge(
    seed_df[['Season', 'TeamID', 'SeedValue']], 
    left_on=['Season', 'TeamID1'], right_on=['Season', 'TeamID'],
    how='left'
).rename(columns={'SeedValue': 'SeedValue1'}).drop(columns=['TeamID'])

# 8. TeamID2에 대한 시드 정보 병합
submission_df = submission_df.merge(
    seed_df[['Season', 'TeamID', 'SeedValue']], 
    left_on=['Season', 'TeamID2'], right_on=['Season', 'TeamID'],
    how='left'
).rename(columns={'SeedValue': 'SeedValue2'}).drop(columns=['TeamID'])

# 9. 결측값 처리 (시드값이 없는 경우 기본값 16으로 설정)
submission_df[['SeedValue1', 'SeedValue2']] = submission_df[['SeedValue1', 'SeedValue2']].fillna(16)

# 10. 시드 차이 계산
submission_df['SeedDiff'] = submission_df['SeedValue1'] - submission_df['SeedValue2']

# 11. 결과 확인
print(submission_df[['ID', 'Season', 'TeamID1', 'TeamID2', 'SeedValue1', 'SeedValue2', 'SeedDiff']].head())

# 12. 최종 데이터 저장
submission_df.to_csv('/kaggle/working/submission_with_seed_diff.csv', index=False)

## #2. 공격 및 수비 효율성 계산(all_team_efficiency)
 - FGA는 필드골 시도, FTA는 자유투 시도, OffReb는 공격 리바운드, TO는 턴오버
 - 공격 효율성 : 팀이 소유한 공격 기회 당 평균 득점
 - Possessions=FGA+0.44×FTA−OffReb+TO
 - OffEff= Total Points Scored/Total Possessions
 - 수비 효율성 : 상대팀의 공격 기회 당 허용 득점
 - OppPossessions=OppFGA+0.44×OppFTA−OppOffReb+OppTO
 - DefEff= Total Points Allowed/Total Opponent Possessions
- 승리한 경기에서는 승리팀으로, 패배한 경기에서는 패배팀으로 등장하므로 두 경우를 처리한 후 팀별로 집계

In [ ]:
# 정규 시즌 상세 결과 데이터 로드
m_detailed_df = pd.read_csv('/kaggle/input/march-machine-learning-mania-2025/MRegularSeasonDetailedResults.csv')
w_detailed_df = pd.read_csv('/kaggle/input/march-machine-learning-mania-2025/WRegularSeasonDetailedResults.csv')

In [ ]:
# 남성 승리한 팀 데이터 생성
m_winners = pd.DataFrame({
    'Season': m_detailed_df['Season'],
    'TeamID': m_detailed_df['WTeamID'],
    'PointsScored': m_detailed_df['WScore'],
    'PointsAllowed': m_detailed_df['LScore'],
    'FGA': m_detailed_df['WFGA'],
    'FTA': m_detailed_df['WFTA'],
    'OffReb': m_detailed_df['WOR'],
    'TO': m_detailed_df['WTO'],
    'OppFGA': m_detailed_df['LFGA'],
    'OppFTA': m_detailed_df['LFTA'],
    'OppOffReb': m_detailed_df['LOR'],
    'OppTO': m_detailed_df['LTO']
})

m_winners['Possessions'] = m_winners['FGA'] + 0.44 * m_winners['FTA'] - m_winners['OffReb'] + m_winners['TO']
m_winners['OppPossessions'] = m_winners['OppFGA'] + 0.44 * m_winners['OppFTA'] - m_winners['OppOffReb'] + m_winners['OppTO']

# 남성 패배한 팀 데이터 생성
m_losers = pd.DataFrame({
    'Season': m_detailed_df['Season'],
    'TeamID': m_detailed_df['LTeamID'],
    'PointsScored': m_detailed_df['LScore'],
    'PointsAllowed': m_detailed_df['WScore'],
    'FGA': m_detailed_df['LFGA'],
    'FTA': m_detailed_df['LFTA'],
    'OffReb': m_detailed_df['LOR'],
    'TO': m_detailed_df['LTO'],
    'OppFGA': m_detailed_df['WFGA'],
    'OppFTA': m_detailed_df['WFTA'],
    'OppOffReb': m_detailed_df['WOR'],
    'OppTO': m_detailed_df['WTO']
})

m_losers['Possessions'] = m_losers['FGA'] + 0.44 * m_losers['FTA'] - m_losers['OffReb'] + m_losers['TO']
m_losers['OppPossessions'] = m_losers['OppFGA'] + 0.44 * m_losers['OppFTA'] - m_losers['OppOffReb'] + m_losers['OppTO']

# 승리와 패배 데이터 결합
m_combined = pd.concat([m_winners, m_losers], axis=0)

# 팀별, 시즌별 집계 (총 득점 및 포제션 합산)
m_team_efficiency = m_combined.groupby(['Season', 'TeamID']).agg({
    'PointsScored': 'sum',
    'Possessions': 'sum',
    'PointsAllowed': 'sum',
    'OppPossessions': 'sum'
}).reset_index()

# 효율성 계산 (남자팀)
m_team_efficiency['OffEff'] = m_team_efficiency['PointsScored'] / m_team_efficiency['Possessions']
m_team_efficiency['DefEff'] = m_team_efficiency['PointsAllowed'] / m_team_efficiency['OppPossessions']

# 무한대 및 결측치 처리
m_team_efficiency.replace([np.inf, -np.inf], np.nan, inplace=True)
m_team_efficiency.fillna(0, inplace=True)

# 남자팀 데이터 저장
m_team_efficiency.to_csv('/kaggle/working/m_team_efficiency.csv', index=False)

In [ ]:
# 여성 승리한 팀 데이터 생성
w_winners = pd.DataFrame({
    'Season': w_detailed_df['Season'],
    'TeamID': w_detailed_df['WTeamID'],
    'PointsScored': w_detailed_df['WScore'],
    'PointsAllowed': w_detailed_df['LScore'],
    'FGA': w_detailed_df['WFGA'],
    'FTA': w_detailed_df['WFTA'],
    'OffReb': w_detailed_df['WOR'],
    'TO': w_detailed_df['WTO'],
    'OppFGA': w_detailed_df['LFGA'],
    'OppFTA': w_detailed_df['LFTA'],
    'OppOffReb': w_detailed_df['LOR'],
    'OppTO': w_detailed_df['LTO']
})

w_winners['Possessions'] = w_winners['FGA'] + 0.44 * w_winners['FTA'] - w_winners['OffReb'] + w_winners['TO']
w_winners['OppPossessions'] = w_winners['OppFGA'] + 0.44 * w_winners['OppFTA'] - w_winners['OppOffReb'] + w_winners['OppTO']

# 여성 패배한 팀 데이터 생성
w_losers = pd.DataFrame({
    'Season': w_detailed_df['Season'],
    'TeamID': w_detailed_df['LTeamID'],
    'PointsScored': w_detailed_df['LScore'],
    'PointsAllowed': w_detailed_df['WScore'],
    'FGA': w_detailed_df['LFGA'],
    'FTA': w_detailed_df['LFTA'],
    'OffReb': w_detailed_df['LOR'],
    'TO': w_detailed_df['LTO'],
    'OppFGA': w_detailed_df['WFGA'],
    'OppFTA': w_detailed_df['WFTA'],
    'OppOffReb': w_detailed_df['WOR'],
    'OppTO': w_detailed_df['WTO']
})

w_losers['Possessions'] = w_losers['FGA'] + 0.44 * w_losers['FTA'] - w_losers['OffReb'] + w_losers['TO']
w_losers['OppPossessions'] = w_losers['OppFGA'] + 0.44 * w_losers['OppFTA'] - w_losers['OppOffReb'] + w_losers['OppTO']

# 승리와 패배 데이터 결합
w_combined = pd.concat([w_winners, w_losers], axis=0)

# 팀별, 시즌별 집계 (총 득점 및 포제션 합산)
w_team_efficiency = w_combined.groupby(['Season', 'TeamID']).agg({
    'PointsScored': 'sum',
    'Possessions': 'sum',
    'PointsAllowed': 'sum',
    'OppPossessions': 'sum'
}).reset_index()

# 효율성 계산
w_team_efficiency['OffEff'] = w_team_efficiency['PointsScored'] / w_team_efficiency['Possessions']
w_team_efficiency['DefEff'] = w_team_efficiency['PointsAllowed'] / w_team_efficiency['OppPossessions']

# 무한대 및 결측치 처리
w_team_efficiency.replace([np.inf, -np.inf], np.nan, inplace=True)
w_team_efficiency.fillna(0, inplace=True)

# 여자팀 데이터 저장
w_team_efficiency.to_csv('/kaggle/working/w_team_efficiency.csv', index=False)

In [ ]:
# Gender 컬럼 추가
m_team_efficiency['Gender'] = 'M'
w_team_efficiency['Gender'] = 'W'

# 남녀 데이터 합치기
all_team_efficiency = pd.concat([m_team_efficiency, w_team_efficiency], ignore_index=True)

# 최종 파일로 저장
all_team_efficiency.to_csv('/kaggle/working/all_team_efficiency.csv', index=False)

## #3. 조정된 효율성 지표(all_team_efficiency)
- 이미 전처리한 팀별 공격 및 수비 효율성 데이터_team_efficiency를 활용함
- 상대 수비력/공격력을 고려한 조정 지표
- 상대 컨퍼런스 강도를 고려한 조정 지표
- 상대 팀의 강도를 고려한 조정된 지표로 모델의 성능을 더 향상시킬 수 있음
- 각 팀이 시즌 동안 경기했던 상대 팀들의 OffEff 및 DefEff의 평균을 계산해야 함
- OpponentOffEfficiency, OpponentDefEfficiency를 경기 상대의 평균으로 조정
- 즉, 특정 팀이 경기한 모든 상대 팀들의 공격/수비 효율을 가져와 평균을 내는 방식

In [ ]:
# 1. 전처리한 팀별 공격 및 수비 효율성 데이터 로드
m_team_efficiency = pd.read_csv('/kaggle/working/m_team_efficiency.csv')  # 남자 팀 데이터
w_team_efficiency = pd.read_csv('/kaggle/working/w_team_efficiency.csv')  # 여자 팀 데이터

# 2. 성별 컬럼 추가 (남자: 'M', 여자: 'W')
m_team_efficiency['Gender'] = 'M'
w_team_efficiency['Gender'] = 'W'

In [ ]:
m_team_efficiency.head()

In [ ]:
# 3. 남녀 데이터 합치기
all_team_efficiency = pd.concat([m_team_efficiency, w_team_efficiency], ignore_index=True)

# 4. NCAA 정규 시즌 경기 데이터 불러오기
m_detailed_results = pd.read_csv('/kaggle/input/march-machine-learning-mania-2025/MRegularSeasonDetailedResults.csv')
w_detailed_results = pd.read_csv('/kaggle/input/march-machine-learning-mania-2025/WRegularSeasonDetailedResults.csv')

# 성별 추가
m_detailed_results['Gender'] = 'M'
w_detailed_results['Gender'] = 'W'

# 남녀 경기 데이터 합기기
all_detailed_results = pd.concat([m_detailed_results, w_detailed_results], ignore_index=True)

# 5. 경기 결과 데이터에서 각 팀의 경기 상대 정보 추출
# 승리한 팀의 경기 기록
winners = all_detailed_results[['Season', 'WTeamID', 'LTeamID', 'Gender']].rename(columns={'WTeamID': 'TeamID', 'LTeamID': 'OpponentID'})
# 패배한 팀의 경기 기록
losers = all_detailed_results[['Season', 'LTeamID', 'WTeamID', 'Gender']].rename(columns={'LTeamID': 'TeamID', 'WTeamID': 'OpponentID'})
# 승리팀과 패배팀 데이터를 합쳐 모든 팀의 경기 내역을 생성
matchups = pd.concat([winners, losers], ignore_index=True)

In [ ]:
print(matchups.columns)
# merge할 때 left 데이터프레임에는 'OffEff'와 'DefEff' 컬럼이 없었기 때문에, 접미사(suffix(_Opponent))가 자동으로 붙지 않아서 KeyError: "Columns not found: 'DefEff_Opponent', 'OffEff_Opponent'"
# 이 컬럼들을 상대 팀의 효율성을 나타내도록 이름을 바꾸면 됨
# merge 후에 컬럼명을 수동으로 변경

In [ ]:
# 1. 상대 팀의 공격/수비 효율성 추가 (merge)
matchups = matchups.merge(
    all_team_efficiency[['Season', 'TeamID', 'OffEff', 'DefEff']],
    left_on=['Season', 'OpponentID'],
    right_on=['Season', 'TeamID'],
    suffixes=('', '_Opponent')
)

# 2. 불필요한 TeamID_Opponent 컬럼 삭제 (병합 후 생성됨)
matchups.drop(columns=['TeamID_Opponent'], inplace=True)

# 3. 오른쪽에서 가져온 OffEff, DefEff 컬럼을 상대 팀의 효율성으로 이름 변경
matchups.rename(columns={'OffEff': 'OffEff_Opponent', 'DefEff': 'DefEff_Opponent'}, inplace=True)

# 4. 이제 상대 팀 평균 효율성을 계산
opponent_avg_efficiency = matchups.groupby(['Season', 'TeamID', 'Gender'])[['OffEff_Opponent', 'DefEff_Opponent']].mean().reset_index()
opponent_avg_efficiency.rename(columns={
    'OffEff_Opponent': 'OpponentOffEfficiency',
    'DefEff_Opponent': 'OpponentDefEfficiency'
}, inplace=True)

# 8. 기존 효율성 데이터와 병합
all_team_efficiency = all_team_efficiency.merge(opponent_avg_efficiency, on=['Season', 'TeamID', 'Gender'], how='left')

# 9. 조정된 효율성 지표 계산
all_team_efficiency['AdjOffEff'] = all_team_efficiency['OffEff'] / all_team_efficiency['OpponentDefEfficiency']
all_team_efficiency['AdjDefEff'] = all_team_efficiency['DefEff'] / all_team_efficiency['OpponentOffEfficiency']

# 10. 결측치 및 무한대 값 처리
all_team_efficiency.replace([np.inf, -np.inf], np.nan, inplace=True)
all_team_efficiency.fillna(0, inplace=True)

# 11. 최종 데이터 저장
all_team_efficiency.to_csv('/kaggle/working/all_adjusted_team_efficiency.csv', index=False)

all_team_efficiency.head()

## #4. 최근 경기 승률(all_team_win_pct)
- 정규 시즌 컴팩트 결과 데이터(RegularSeasonCompactResults)에서 가장 최근 시즌 혹은 관심 시즌 데이터를 선택하여 승률을 기록을 확인
- 팀별 총 경기 수와 평균 승률 계산
- 남/녀 각각 데이터 전처리 후 하나의 파일 필요
- 히스토그램을 통해 승률 분포를 확인하고, 산점도를 통해 팀별 경기 수와 승률의 관계를 시각화

In [ ]:
import pandas as pd

# 1. 남녀 정규 시즌 컴팩트 결과 데이터 불러오기
m_compact = pd.read_csv('/kaggle/input/march-machine-learning-mania-2025/MRegularSeasonCompactResults.csv')
w_compact = pd.read_csv('/kaggle/input/march-machine-learning-mania-2025/WRegularSeasonCompactResults.csv')

# 2. 성별 컬럼 추가 (남자: 'M', 여자: 'W')
m_compact['Gender'] = 'M'
w_compact['Gender'] = 'W'

# 3. 남자 데이터 처리
# 최신 시즌 선택
current_season_m = m_compact['Season'].max()
recent_games_m = m_compact[m_compact['Season'] == current_season_m]

# 승리 팀 데이터 (승리: Win=1)
m_wins = recent_games_m[['Season', 'WTeamID', 'Gender']].copy()
m_wins['Win'] = 1
m_wins.rename(columns={'WTeamID': 'TeamID'}, inplace=True)

# 패배 팀 데이터 (패배: Win=0)
m_losses = recent_games_m[['Season', 'LTeamID', 'Gender']].copy()
m_losses['Win'] = 0
m_losses.rename(columns={'LTeamID': 'TeamID'}, inplace=True)

# 승패 데이터를 하나의 DataFrame으로 결합
m_team_results = pd.concat([m_wins, m_losses], ignore_index=True)

# 팀별, 시즌별로 승률과 경기 수 계산
m_team_win_pct = m_team_results.groupby(['Season', 'TeamID', 'Gender'])['Win'].agg(['mean', 'count']).reset_index()
m_team_win_pct.rename(columns={'mean': 'RecentWinPct', 'count': 'GamesPlayed'}, inplace=True)

m_team_win_pct.columns

In [ ]:
# 4. 여자 데이터 처리
# 최신 시즌 선택
current_season_w = w_compact['Season'].max()
recent_games_w = w_compact[w_compact['Season'] == current_season_w]

# 승리 팀 데이터 (승리: Win=1)
w_wins = recent_games_w[['Season', 'WTeamID', 'Gender']].copy()
w_wins['Win'] = 1
w_wins.rename(columns={'WTeamID': 'TeamID'}, inplace=True)

# 패배 팀 데이터 (패배: Win=0)
w_losses = recent_games_w[['Season', 'LTeamID', 'Gender']].copy()
w_losses['Win'] = 0
w_losses.rename(columns={'LTeamID': 'TeamID'}, inplace=True)

# 승패 데이터를 하나의 DataFrame으로 결합
w_team_results = pd.concat([w_wins, w_losses], ignore_index=True)

# 팀별, 시즌별로 승률과 경기 수 계산
w_team_win_pct = w_team_results.groupby(['Season', 'TeamID', 'Gender'])['Win'].agg(['mean', 'count']).reset_index()
w_team_win_pct.rename(columns={'mean': 'RecentWinPct', 'count': 'GamesPlayed'}, inplace=True)

# 5. 남녀 데이터 통합
all_team_win_pct = pd.concat([m_team_win_pct, w_team_win_pct], ignore_index=True)

# 6. 최종 데이터 저장
all_team_win_pct.to_csv('/kaggle/working/all_recent_win_pct.csv', index=False)

all_team_win_pct.head()

## #5. 토너먼트 경험 및 과거 성적(tournament_stats)
- 아래의 새로운 피처를 생성해야 함
- #1. TournamentAppearances (토너먼트 진출 횟수)
     - 과거 몇 년 동안 토너먼트에 진출했었는지
- #2. BestTournamentFinish (최고 성적)
     - 과거 가장 좋은 성적을 거둔 라운드 확인
- #3. RecentTournamentPerformance (최근 몇 시즌 성적)
     - 최근 몇 년간 토너먼트에서 평균적으로 몇 라운드까지 진출했는지
- 모델이 팀의 토너먼트 경험을 학습하는 데 도움을 줄 수 있음

In [ ]:
# 1. NCAA 토너먼트 경기 데이터 불러오기
m_tourney_results = pd.read_csv('/kaggle/input/march-machine-learning-mania-2025/MNCAATourneyCompactResults.csv')
w_tourney_results = pd.read_csv('/kaggle/input/march-machine-learning-mania-2025/WNCAATourneyCompactResults.csv')

# 2. 성별 컬럼 추가
m_tourney_results['Gender'] = 'M'
w_tourney_results['Gender'] = 'W'

# 3. 남녀 데이터 통합
tourney_results = pd.concat([m_tourney_results, w_tourney_results], ignore_index=True)

# 4. 토너먼트 진출 횟수 계산
tourney_appearances = pd.concat([
    tourney_results[['Season', 'WTeamID', 'Gender']].rename(columns={'WTeamID': 'TeamID'}),
    tourney_results[['Season', 'LTeamID', 'Gender']].rename(columns={'LTeamID': 'TeamID'})
])
tourney_appearances = tourney_appearances.groupby(['TeamID', 'Gender'])['Season'].nunique().reset_index()
tourney_appearances.rename(columns={'Season': 'TournamentAppearances'}, inplace=True)

# 5. 각 팀의 최고 성적 계산
tourney_results['Round'] = pd.cut(
    tourney_results['DayNum'],
    bins=[0, 136, 138, 143, 145, 152, 154],
    labels=['First Round', 'Second Round', 'Sweet 16', 'Elite 8', 'Final Four', 'Championship'],
    ordered=True
)

best_finish = pd.concat([
    tourney_results[['Season', 'WTeamID', 'Round', 'Gender']].rename(columns={'WTeamID': 'TeamID'}),
    tourney_results[['Season', 'LTeamID', 'Round', 'Gender']].rename(columns={'LTeamID': 'TeamID'})
])


# # 최고 성적을 계산할 때 정렬된 categorical 순서를 유지
# best_finish['Round'] = best_finish['Round'].astype(pd.CategoricalDtype(
#     categories=['First Round', 'Second Round', 'Sweet 16', 'Elite 8', 'Final Four', 'Championship'],
#     ordered=True
# )) //typyerror 'Round'열을 Categofical 타입으로 변환했는는데, None(결측값)이 포함되어있어 새로운 카테고리에 추가 못함

# 1). Round 컬럼을 Categorical 타입으로 변환하기 전에 결측값을 'None'으로 채우기
# 결측값을 'None'으로 처리 후 문자열로 변환
best_finish['Round'] = best_finish['Round'].astype(str).fillna('None')
# 2) Categorical 변환 (순서 유지)
round_categories = ['None', 'First Round', 'Second Round', 'Sweet 16', 'Elite 8', 'Final Four', 'Championship']
best_finish['Round'] = best_finish['Round'].astype(pd.CategoricalDtype(categories=round_categories, ordered=True))
# 3) 숫자로 변환
best_finish['RoundCode'] = best_finish['Round'].cat.codes
# 4) 그룹화하여 최고 진출 라운드의 코드 추출
best_finish = best_finish.groupby(['TeamID', 'Gender'])['RoundCode'].max().reset_index()
# 5) 코드 값을 다시 라벨로 매핑
best_finish['BestTournamentFinish'] = best_finish['RoundCode'].apply(lambda x: round_categories[x])
best_finish.drop(columns=['RoundCode'], inplace=True)

# 6. 최근 5시즌 평균 성적 계산
recent_seasons = 5
latest_season = tourney_results['Season'].max()
recent_performance = tourney_results[tourney_results['Season'] >= latest_season - recent_seasons]
recent_performance = pd.concat([
    recent_performance[['Season', 'WTeamID', 'Round', 'Gender']].rename(columns={'WTeamID': 'TeamID'}),
    recent_performance[['Season', 'LTeamID', 'Round', 'Gender']].rename(columns={'LTeamID': 'TeamID'})
])
# Convert Round to numeric codes and compute average
recent_performance['Round'] = recent_performance['Round'].cat.codes
recent_performance = recent_performance.groupby(['TeamID', 'Gender'])['Round'].mean().reset_index()
recent_performance.rename(columns={'Round': 'RecentTournamentPerformance'}, inplace=True)

# 7. 모든 토너먼트 관련 데이터를 하나의 데이터프레임으로 결합
tournament_stats = tourney_appearances.merge(best_finish, on=['TeamID', 'Gender'], how='left')
tournament_stats = tournament_stats.merge(recent_performance, on=['TeamID', 'Gender'], how='left')

# 8. 결측치 처리
tournament_stats.fillna({
    'TournamentAppearances': 0,
    'BestTournamentFinish': 'None',
    'RecentTournamentPerformance': 0
}, inplace=True)
# 시즌 정보 추가 (최신 시즌 기준으로 채우기)
tournament_stats['Season'] = latest_season

column_order = ['Season', 'TeamID', 'Gender', 'TournamentAppearances', 'BestTournamentFinish', 'RecentTournamentPerformance']
tournament_stats = tournament_stats[column_order]

# 9. 최종 데이터 저장
tournament_stats.to_csv('/kaggle/working/all_tournament_stats.csv', index=False)

tournament_stats.head()
# 'Season' 칼럼이 없음 KeyError 예상 - 적절한 값으로 채워줘야 함

## #6. 컨퍼런스 경쟁력 및 대회 성적(team_conference_strength)

- ConferenceAvgSeasonWinPct : 해당 컨퍼런스의 모든 팀이 정규 시즌에서 기록한 평균 승률 - 컨퍼런스 내 팀들이 정규 시즌동안 얼마나 경쟁력이 있는지
- ConferenceTournamentWinPct : 컨퍼런스 별 NCAA토너먼트에서의 평균 승률 - 해당 컨퍼런스가 토너먼트에서 강한 컨퍼런스인지 판단
- ConferenceAvgTournamentRound : 컨퍼런스 내 팀들이 토너먼트에서 평균적으로 어느 라운드까지 진출하는지
- ConferencePowerIndex (CPI, 가중평균) : 위 3가지 지표를 결합한 종합적인 지표

In [ ]:
# 1. 데이터 로드 
# Conferences 데이터 (참고용; 실제 병합에는 M/W 팀 컨퍼런스 파일 사용)
conferences_df = pd.read_csv('/kaggle/input/march-machine-learning-mania-2025/Conferences.csv')
# M/W Team Conferences 데이터
m_team_conferences = pd.read_csv('/kaggle/input/march-machine-learning-mania-2025/MTeamConferences.csv')
w_team_conferences = pd.read_csv('/kaggle/input/march-machine-learning-mania-2025/WTeamConferences.csv')
# 성별 컬럼 추가 
m_team_conferences['Gender'] = 'M'
w_team_conferences['Gender'] = 'W'
# 남녀 팀 컨퍼런스 데이터 합치기 (병합에 사용할 컬럼: Season, TeamID, ConfAbbrev, Gender)
team_conferences = pd.concat([m_team_conferences, w_team_conferences], ignore_index=True)
team_conferences = team_conferences[['Season', 'TeamID', 'ConfAbbrev', 'Gender']]


# M/W 정규 시즌 컴팩트 결과 데이터 로드
m_regular_season = pd.read_csv('/kaggle/input/march-machine-learning-mania-2025/MRegularSeasonCompactResults.csv')
w_regular_season = pd.read_csv('/kaggle/input/march-machine-learning-mania-2025/WRegularSeasonCompactResults.csv')
# 성별 칼럼 추가 
m_regular_season['Gender'] = 'M'
w_regular_season['Gender'] = 'W'
# 남녀 팀 정규 데이터 합치기 
regular_season = pd.concat([m_regular_season, w_regular_season], ignore_index=True)

# M/W 토너먼트 컴팩트 결과 데이터 로드
m_tourney_results = pd.read_csv('/kaggle/input/march-machine-learning-mania-2025/MNCAATourneyCompactResults.csv')
w_tourney_results = pd.read_csv('/kaggle/input/march-machine-learning-mania-2025/WNCAATourneyCompactResults.csv')
# 성별 칼럼 추가 
m_tourney_results['Gender'] = 'M'
w_tourney_results['Gender'] = 'W'
# 남녀 팀 컴팩트 결과 데이터 합기기 
# 추후 코칭 요소 데이터에서 다시 전처리 들어감
tourney_results = pd.concat([m_tourney_results, w_tourney_results], ignore_index=True)

In [ ]:
# 2. 정규 시즌 컨퍼런스 승률 계산
# 승리 데이터: 승리팀은 Win=1, TeamID를 WTeamID로 변경
wins = regular_season[['Season', 'WTeamID', 'Gender']].copy()
wins['Win'] = 1
wins.rename(columns={'WTeamID': 'TeamID'}, inplace=True)

# 패배 데이터: 패배팀은 Win=0, TeamID를 LTeamID로 변경
losses = regular_season[['Season', 'LTeamID', 'Gender']].copy()
losses['Win'] = 0
losses.rename(columns={'LTeamID': 'TeamID'}, inplace=True)

# 승패 데이터를 하나의 DataFrame으로 결합
season_results = pd.concat([wins, losses], ignore_index=True)

# team_conferences와 병합하여 각 팀의 컨퍼런스 정보 추가
season_results = season_results.merge(team_conferences, on=['Season', 'TeamID'], how='left', suffixes=('', '_conf'))
# 중복된 Gender 컬럼이 발생할 경우, 기존 값을 우선 사용
season_results['Gender'] = season_results['Gender'].fillna(season_results['Gender_conf'])
season_results.drop(columns=['Gender_conf'], inplace=True)

# 팀별, 시즌별 컨퍼런스 평균 승률 계산
conference_season_win_pct = season_results.groupby(['Season', 'ConfAbbrev', 'Gender'])['Win'].mean().reset_index()
conference_season_win_pct.rename(columns={'Win': 'ConferenceAvgSeasonWinPct'}, inplace=True)

In [ ]:
# 3. NCAA 토너먼트 컨퍼런스 승률 계산

# 승리팀의 컨퍼런스 정보 병합
tourney_results = tourney_results.merge(
    team_conferences, left_on=['Season', 'WTeamID'], right_on=['Season', 'TeamID'],
    how='left'
)
tourney_results.rename(columns={'ConfAbbrev': 'WConfAbbrev'}, inplace=True)
tourney_results.drop(columns=['TeamID'], inplace=True)  # 중복 제거

# 패배팀의 컨퍼런스 정보 병합
tourney_results = tourney_results.merge(
    team_conferences, left_on=['Season', 'LTeamID'], right_on=['Season', 'TeamID'],
    how='left'
)
tourney_results.rename(columns={'ConfAbbrev': 'LConfAbbrev'}, inplace=True)
tourney_results.drop(columns=['TeamID'], inplace=True)

# 승리/패배별 그룹화하여 컨퍼런스 승/패 횟수 계산
conf_wins = tourney_results.groupby(['Season', 'WConfAbbrev', 'Gender']).size().reset_index(name='Wins')
conf_losses = tourney_results.groupby(['Season', 'LConfAbbrev', 'Gender']).size().reset_index(name='Losses')

# 두 데이터를 outer join 하여 토너먼트 승률 계산
conf_tourney_win_rate = conf_wins.merge(
    conf_losses,
    left_on=['Season', 'WConfAbbrev', 'Gender'],
    right_on=['Season', 'LConfAbbrev', 'Gender'],
    how='outer'
).fillna(0)

conf_tourney_win_rate['TotalGames'] = conf_tourney_win_rate['Wins'] + conf_tourney_win_rate['Losses']
conf_tourney_win_rate['ConferenceTournamentWinPct'] = conf_tourney_win_rate['Wins'] / conf_tourney_win_rate['TotalGames']

conf_tourney_win_rate = conf_tourney_win_rate[['Season', 'WConfAbbrev', 'Gender', 'ConferenceTournamentWinPct']]
conf_tourney_win_rate.rename(columns={'WConfAbbrev': 'ConfAbbrev'}, inplace=True)

In [ ]:
# 4. NCAA 토너먼트 평균 진출 라운드 계산

# 토너먼트 데이터에서 DayNum을 기준으로 라운드 분류
tourney_results['Round'] = pd.cut(
    tourney_results['DayNum'],
    bins=[0, 136, 138, 143, 145, 152, 154],
    labels=['First Round', 'Second Round', 'Sweet 16', 'Elite 8', 'Final Four', 'Championship']
)

# 승리팀과 패배팀의 데이터를 각각 TeamID로 통일 후 결합
conf_avg_round = pd.concat([
    tourney_results[['Season', 'WConfAbbrev', 'Round', 'Gender']].rename(columns={'WConfAbbrev': 'ConfAbbrev'}),
    tourney_results[['Season', 'LConfAbbrev', 'Round', 'Gender']].rename(columns={'LConfAbbrev': 'ConfAbbrev'})
], ignore_index=True)

# 그룹별로 토너먼트 진출 라운드를 숫자로 변환하여 평균 계산
conf_avg_round = conf_avg_round.groupby(['Season', 'ConfAbbrev', 'Gender'])['Round'].apply(
    lambda x: x.cat.codes.mean()
).reset_index()
conf_avg_round.rename(columns={'Round': 'ConferenceAvgTournamentRound'}, inplace=True)

In [ ]:
# 5. 컨퍼런스 강도 지수 (Conference Power Index, CPI) 계산

# 위의 데이터들을 공통 키(Season, ConfAbbrev, Gender)로 병합
conference_strength = conference_season_win_pct.merge(
    conf_tourney_win_rate, on=['Season', 'ConfAbbrev', 'Gender'], how='left'
)
conference_strength = conference_strength.merge(
    conf_avg_round, on=['Season', 'ConfAbbrev', 'Gender'], how='left'
)

# CPI 계산: 가중치를 부여하여 통합
conference_strength['ConferencePowerIndex'] = (
    (conference_strength['ConferenceAvgSeasonWinPct'] * 0.4) +
    (conference_strength['ConferenceTournamentWinPct'] * 0.4) +
    (conference_strength['ConferenceAvgTournamentRound'] * 0.2)
)

# TeamID KeyError... 
# 1.TeamID를 추가하여 conference_strength와 병합할 준비
team_conference_strength = team_conferences.merge(
    conference_strength, on=['Season', 'ConfAbbrev', 'Gender'], how='left'
)

# 2.결측치 처리 (만약 병합 후 값이 없는 경우)
team_conference_strength.fillna({
    'ConferenceTournamentWinPct': 0, 
    'ConferenceAvgTournamentRound': 0, 
    'ConferencePowerIndex': 0
}, inplace=True)

# 3. 병합 후 최종 저장
team_conference_strength.to_csv('/kaggle/working/all_team_conference_strength.csv', index=False)

team_conference_strength.head()

## #7. 팀별 경기 속도 및 스타일 (all_team_stats)
- 남/녀 정규 시즌 상세 데이터_Regular Season Detailed Results를 활용하여 각 팀의 경기 속도와 공격 스타일을 계산
- 공격 스타일 : 3점슛 시도 비율, 페인트존 공격 빈도
- 수비 스타일 : 압박 수비 여부, 턴오버 유도율

In [ ]:
# 1. MRegularSeasonDetailedResults & WRegularSeasonDetailedResults 데이터
m_detailed_df = pd.read_csv('/kaggle/input/march-machine-learning-mania-2025/MRegularSeasonDetailedResults.csv')
w_detailed_df = pd.read_csv('/kaggle/input/march-machine-learning-mania-2025/WRegularSeasonDetailedResults.csv')

# 성별 컬럼 추가
m_detailed_df['Gender'] = 'M'
w_detailed_df['Gender'] = 'W'

m_detailed_df.head()

In [ ]:
# 2. Function to Process Team Statistics
def process_team_stats(detailed_df):
    # 승리한 팀 데이터
    winners = pd.DataFrame({
        'Season': detailed_df['Season'],
        'TeamID': detailed_df['WTeamID'],
        'FGA': detailed_df['WFGA'],
        'FTA': detailed_df['WFTA'],
        'OffReb': detailed_df['WOR'],
        'TO': detailed_df['WTO'],
        'FGA3': detailed_df['WFGA3'],
        'Gender': detailed_df['Gender']
    })
    winners['Possessions'] = winners['FGA'] + 0.44 * winners['FTA'] - winners['OffReb'] + winners['TO']

    # 패배한 팀 데이터
    losers = pd.DataFrame({
        'Season': detailed_df['Season'],
        'TeamID': detailed_df['LTeamID'],
        'FGA': detailed_df['LFGA'],
        'FTA': detailed_df['LFTA'],
        'OffReb': detailed_df['LOR'],
        'TO': detailed_df['LTO'],
        'FGA3': detailed_df['LFGA3'],
        'Gender': detailed_df['Gender']
    })
    losers['Possessions'] = losers['FGA'] + 0.44 * losers['FTA'] - losers['OffReb'] + losers['TO']

    # 승리 & 패배 데이터 결합
    combined = pd.concat([winners, losers], ignore_index=True)

    # 3. 팀별 시즌별 집계
    team_stats = combined.groupby(['Season', 'TeamID', 'Gender']).agg({
        'Possessions': 'sum',
        'FGA': 'sum',
        'FTA': 'sum',
        'FGA3': 'sum',
        'Season': 'count'
    }).rename(columns={'Season': 'GamesPlayed'}).reset_index()

    # 4. 주요 지표 계산
    team_stats['Pace'] = team_stats['Possessions'] / team_stats['GamesPlayed']
    team_stats['ThreePtAttemptRatio'] = team_stats['FGA3'] / team_stats['FGA']
    team_stats['FreeThrowRate'] = team_stats['FTA'] / team_stats['FGA']

    # 5. 결측값 및 무한대 처리
    team_stats.replace([np.inf, -np.inf], np.nan, inplace=True)
    team_stats.fillna(0, inplace=True)

    return team_stats

# 6. 남녀 데이터 처리
m_team_stats = process_team_stats(m_detailed_df)
w_team_stats = process_team_stats(w_detailed_df)

# 7. 남/녀 데이터를 하나로 합침
all_team_stats = pd.concat([m_team_stats, w_team_stats], ignore_index=True)

# 8. 통합 데이터 저장
all_team_stats.to_csv('/kaggle/working/all_team_pace_and_style.csv', index=False)

## #8. 홈/ 원정/ 중립 경기 승률 차이(location_win_pct)
 - NCAA 토너먼트에서는 중립 경기(WLoc = N)가 많기 때문에, 토너먼트 성적과 연관될 가능성이 높음
 - 홈(H), 원정(A), 중립(N) 경기별 승률 계산
 - 홈-원정 승률 차이 (Home Court Advantage) 분석
 - 원정 승률-NCAA 토너먼트 성적

In [ ]:
# 1. 남녀 정규 시즌 컴팩트 결과 데이터
m_compact = pd.read_csv('/kaggle/input/march-machine-learning-mania-2025/MRegularSeasonCompactResults.csv')
w_compact = pd.read_csv('/kaggle/input/march-machine-learning-mania-2025/WRegularSeasonCompactResults.csv')

# 2. 성별 컬럼 추가
m_compact['Gender'] = 'M'
w_compact['Gender'] = 'W'

# 3. 남녀 데이터 합치기
regular_season = pd.concat([m_compact, w_compact], ignore_index=True)

# 4. 승리/패배 데이터 만들기
wins = regular_season[['Season', 'WTeamID', 'WLoc', 'Gender']].copy()
wins['Win'] = 1
wins.rename(columns={'WTeamID': 'TeamID'}, inplace=True)

losses = regular_season[['Season', 'LTeamID', 'WLoc', 'Gender']].copy()
losses['Win'] = 0
losses.rename(columns={'LTeamID': 'TeamID'}, inplace=True)

# 5. 승패 기록 결합
team_results = pd.concat([wins, losses], ignore_index=True)

# 6. 경기 장소별 승률 계산
location_win_pct = team_results.groupby(['Season', 'TeamID', 'WLoc', 'Gender'])['Win'].mean().unstack().reset_index()

# 7. 컬럼 이름 변경
location_win_pct.rename(columns={'H': 'HomeWinPct', 'A': 'AwayWinPct', 'N': 'NeutralWinPct'}, inplace=True)

# 8. 결측값(NaN) 처리 → 한 번도 해당 위치에서 경기하지 않은 경우 0으로 채움
location_win_pct.fillna(0, inplace=True)

# 9. 데이터 저장
location_win_pct.to_csv('/kaggle/working/team_location_win_pct.csv', index=False)

location_win_pct.head()

## #9. 개별 선수 및 코칭 요소(coach_stats)
- 주요 선수 부상 여부
- 복귀한 주요 선수의 영향
- 코치 전략 : NCAA에서의 과거 성적, 특정 경기 스타일 선호 여부

In [ ]:
# NCAA 코치 데이터
coaches = pd.read_csv('/kaggle/input/march-machine-learning-mania-2025/MTeamCoaches.csv')
# 필요한 컬럼만 남기기 (Season, TeamID, CoachName)
coaches['Gender'] = 'M'  
coaches = coaches[['Season', 'TeamID', 'Gender', 'CoachName']]

# 저장
coaches.to_csv('/kaggle/working/processed_coach_data.csv', index=False)
coaches.head()

In [ ]:
# 승리팀과 패배팀 데이터를 각각 TeamID로 통일하여 결합
# 승리팀 데이터: Win=1
winners = tourney_results[['Season', 'WTeamID', 'DayNum', 'Gender']].copy()
winners['Win'] = 1
winners.rename(columns={'WTeamID': 'TeamID'}, inplace=True)

# 패배팀 데이터: Win=0
losers = tourney_results[['Season', 'LTeamID', 'DayNum', 'Gender']].copy()
losers['Win'] = 0
losers.rename(columns={'LTeamID': 'TeamID'}, inplace=True)

# 두 데이터를 결합하여 tourney_team_results 생성
tourney_team_results = pd.concat([winners, losers], ignore_index=True)

In [ ]:
# !!!!! 처음부터 모든 데이터 전처리 과정에서 칼럼 리스트들 통일 했어야 했다... Gender, TeamID 아아앙아아ㅏㅏㅏㄱ KeyError...


# 1. NCAA 토너먼트 팀 데이터 (승리/패배 팀 데이터를 미리 준비했다고 가정)
# 예시: tourney_team_results는 ['Season', 'TeamID', 'DayNum', 'Win', 'Gender'] 컬럼을 갖고 있음

# 코치 성과 데이터 불러오기
coach_performance = pd.read_csv('/kaggle/working/coach_performance_raw.csv')

# Gender 정리 (Gender_tourney와 Gender_coach를 하나로 통합)
coach_performance['Gender'] = coach_performance['Gender_tourney'].combine_first(coach_performance['Gender_coach'])

# 필요 없는 Gender 관련 컬럼 삭제
coach_performance.drop(columns=['Gender_tourney', 'Gender_coach'], inplace=True)

# 토너먼트 진출 라운드 계산 (DayNum 기준 변환)
coach_performance['Round'] = pd.cut(
    coach_performance['DayNum'],
    bins=[0, 136, 138, 143, 145, 152, 154],
    labels=['First Round', 'Second Round', 'Sweet 16', 'Elite 8', 'Final Four', 'Championship'],
    ordered=True
)

# 코치별 평균 NCAA 토너먼트 진출 라운드 계산
coach_avg_round = coach_performance.groupby(['Season', 'TeamID', 'Gender'])['Round'].apply(
    lambda x: x.cat.codes.mean() if not x.isnull().all() else 0
).reset_index()
coach_avg_round.rename(columns={'Round': 'AvgTournamentRound'}, inplace=True)

# 코치별 NCAA 토너먼트 승률 계산
coach_win_rate = coach_performance.groupby(['Season', 'TeamID', 'Gender'])['Win'].mean().reset_index()
coach_win_rate.rename(columns={'Win': 'CoachTournamentWinPct'}, inplace=True)

# 코치 성과 데이터 병합 (Season, TeamID, Gender 기준)
coach_stats = coach_avg_round.merge(coach_win_rate, on=['Season', 'TeamID', 'Gender'], how='left')

# 결측값 처리
coach_stats.fillna({'AvgTournamentRound': 0, 'CoachTournamentWinPct': 0}, inplace=True)

# 최종 데이터 저장
coach_stats.to_csv('/kaggle/working/coach_stats.csv', index=False)

In [ ]:
# # Gender_x와 Gender_y가 모두 존재하므로, 결측치가 있는 경우 한 쪽 값을 사용
coach_performance['Gender'] = coach_performance['Gender_x'].combine_first(coach_performance['Gender_y'])

# # 불필요한 중복 컬럼 제거
coach_performance.drop(columns=['Gender_x', 'Gender_y'], inplace=True)

# # 확인
# print(coach_performance.columns)

## #10. 지역별 원정 경기 승률(location_win_pct)
- 홈경기(Home), 원정 경기(Away), 중립 경기(Neutral Performance)의 성적 비교
- 홈-원정 승률 차이 (Home Court Advantage) 분석(특정 지역에서 강한 팀과 약한 팀을 알 수 있음)
- 원정 승률과 NCAA 토너먼트 성적 간의 상관관계 분석
- 최종 데이터셋에 홈/원정/중립 승률 추가하여 저장
- 이 승률 데이터로 홈 어드벤티지 분석 가능 : 홈-원정 승률
- 홈-원정 승률 차이(Home Court Advantage)는 홈 경기가 유리 한지, 원정 경기에서 유리한 팀인지를 알 수 있는 데이터
- NCAA 토너먼트는 중립(N)으로 치러지므로, 원정 승률이 높은 팀이 유리할 수 있음

In [ ]:
# 1. 정규 시즌 결과 데이터 불러오기
m_compact = pd.read_csv('/kaggle/input/march-machine-learning-mania-2025/MRegularSeasonCompactResults.csv')
w_compact = pd.read_csv('/kaggle/input/march-machine-learning-mania-2025/WRegularSeasonCompactResults.csv')

# 2. 성별 컬럼 추가
m_compact['Gender'] = 'M'
w_compact['Gender'] = 'W'

# 3. 남녀 데이터 합치기
regular_season = pd.concat([m_compact, w_compact], ignore_index=True)

# 4. 승리/패배 데이터 만들기
# 승리한 팀은 Win=1, 패배한 팀은 Win=0으로 설정하며, 팀 ID를 통일하기 위해 컬럼명을 변경합니다.
wins = regular_season[['Season', 'WTeamID', 'WLoc', 'Gender']].copy()
wins['Win'] = 1
wins.rename(columns={'WTeamID': 'TeamID'}, inplace=True)

losses = regular_season[['Season', 'LTeamID', 'WLoc', 'Gender']].copy()
losses['Win'] = 0
losses.rename(columns={'LTeamID': 'TeamID'}, inplace=True)

# 5. 승패 기록 결합
team_results = pd.concat([wins, losses], ignore_index=True)

# 6. 경기 장소별 승률 계산
# 'WLoc' 값을 기준으로 pivot_table을 만들어, 각 팀별로 홈(H), 원정(A), 중립(N) 승률을 계산합니다.
location_win_pct = team_results.pivot_table(
    index=['Season', 'TeamID', 'Gender'], 
    columns='WLoc', 
    values='Win', 
    aggfunc='mean'
).reset_index()

# 7. 컬럼 이름 변경: H(HomeWinPct), A(AwayWinPct), N(NeutralWinPct)
location_win_pct.rename(columns={'H': 'HomeWinPct', 'A': 'AwayWinPct', 'N': 'NeutralWinPct'}, inplace=True)

# 8. 결측값 처리: 해당 경기 장소에서 경기를 치르지 않은 경우 NaN 값을 0으로 채움
location_win_pct.fillna(0, inplace=True)

# 9. 최종 데이터 저장
location_win_pct.to_csv('/kaggle/working/all_team_location_win_pct.csv', index=False)

location_win_pct.head()

In [ ]:
# 1. 홈-원정 승률 차이 계산
location_win_pct['HomeAwayWinDiff'] = location_win_pct['HomeWinPct'] - location_win_pct['AwayWinPct']

# 2. 홈이 유리한지, 원정에서도 강한 팀인지 분석
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
sns.histplot(location_win_pct['HomeAwayWinDiff'], bins=20, kde=True, color='dodgerblue')
plt.axvline(0, color='red', linestyle='--', label="Equal Home & Away Win Rate")
plt.title("Home Court Advantage (Home Win % - Away Win %)")
plt.xlabel("Home Win % - Away Win %")
plt.ylabel("Number of Teams")
plt.legend()
plt.show()

## <홈-원정 승률 차이 히스토그램 해석>
- x축: 홈 승률-원정 승률 = HomeAwayWinDiff
- y축: 차이를 갖는 팀의 개수 = Number of Teams
- x축 값이 양수 쪽으로 많으면, 홈에서 유리한 팀들이 많음을 의미 = 홈-원정 데이터 유의미함
- x=0을 기준으로 팀들의 분포를 비교하면 홈 어드밴티지 존재하는지 시각적으로 확인할 수 있겠다 생각함 = 홈과 원정 승률이 같은 팀의 기준선 역할(의미있나 싶긴함)
- 0보다 크면 홈 승률이 더 높음
- 0보다 작으면 원정 승률이 더 높음
- plt.axvline(0, color='red', linestyle='--', label="Equal Home & Away Win Rate")
- 그래프를 보면 데이터가 거의 정규 분포 형태, 이런 경우 정규분포 가정(데이터 변환) 필요 없음
- 이상치(Outliers) 확인 가능 : 홈(오른쪽), 원정(왼쪽)에서만 극단적으로 강한 팀 파악 (양 끝 파악)

# Correlation Analysis

In [ ]:
# 1. NCAA 토너먼트 데이터 불러오기
m_tourney = pd.read_csv('/kaggle/input/march-machine-learning-mania-2025/MNCAATourneyCompactResults.csv')
w_tourney = pd.read_csv('/kaggle/input/march-machine-learning-mania-2025/WNCAATourneyCompactResults.csv')

# 2. 성별 추가 후 합치기
m_tourney['Gender'] = 'M'
w_tourney['Gender'] = 'W'
tourney_results = pd.concat([m_tourney, w_tourney], ignore_index=True)

# 3. 각 팀이 NCAA 토너먼트에서 얼마나 멀리 갔는지 라운드 정보 추가
tourney_results['Round'] = pd.cut(
    tourney_results['DayNum'],
    bins=[0, 136, 138, 143, 145, 152, 154],
    labels=['First Round', 'Second Round', 'Sweet 16', 'Elite 8', 'Final Four', 'Championship']
)

# 4. 최종 진출 라운드 계산
tourney_performance = pd.concat([
    tourney_results[['Season', 'WTeamID', 'Round', 'Gender']].rename(columns={'WTeamID': 'TeamID'}),
    tourney_results[['Season', 'LTeamID', 'Round', 'Gender']].rename(columns={'LTeamID': 'TeamID'})
])
tourney_performance = tourney_performance.groupby(['Season', 'TeamID', 'Gender'])['Round'].apply(lambda x: x.cat.codes.max()).reset_index()
tourney_performance.rename(columns={'Round': 'MaxTournamentRound'}, inplace=True)

# 5. 원정 승률과 NCAA 성적 결합
tourney_performance = tourney_performance.merge(location_win_pct, on=['Season', 'TeamID', 'Gender'], how='left')

# 6. 상관관계 분석
corr_matrix = tourney_performance[['AwayWinPct', 'HomeWinPct', 'NeutralWinPct', 'MaxTournamentRound']].corr()
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5)
plt.title("Correlation: NCAA Tournament Performance vs Home/Away/Neutral Win Rate")
plt.show()

- HomeWinPct, AwayWinPct : 토너먼트 성적 예측에 유용할 가능성이 있음
- NeutralWinPct : NCAA 토너먼트 성적과 관계가 거의 없으므로 모델 변수에서 제외하는 것도 고려

# final file 

In [ ]:
# 1. 데이터 불러오기
submission = pd.read_csv('/kaggle/working/submission_with_seed_diff.csv')  # 시드 차이 데이터
location_win = pd.read_csv('/kaggle/working/all_team_location_win_pct.csv')  # 홈/원정 승률 데이터
efficiency = pd.read_csv('/kaggle/working/all_team_efficiency.csv')  # 팀별 효율성 데이터
adjusted_efficiency = pd.read_csv('/kaggle/working/all_adjusted_team_efficiency.csv')  # 조정된 효율성 데이터
win_pct = pd.read_csv('/kaggle/working/all_recent_win_pct.csv')  # 최근 승률 데이터
tournament_stats = pd.read_csv('/kaggle/working/all_tournament_stats.csv')  # 토너먼트 경험 데이터
team_conference_strength = pd.read_csv('/kaggle/working/all_team_conference_strength.csv')  # 컨퍼런스 강도 데이터
pace_style = pd.read_csv('/kaggle/working/all_team_pace_and_style.csv')  # 경기 스타일 및 템포 데이터

# 2. 데이터 결합 (Season, TeamID, Gender 기준)
merged_df = efficiency.merge(win_pct, on=['Season', 'TeamID', 'Gender'], how='left')
merged_df = merged_df.merge(tournament_stats, on=['Season', 'TeamID', 'Gender'], how='left')
merged_df = merged_df.merge(team_conference_strength, on=['Season', 'TeamID', 'Gender'], how='left')
merged_df = merged_df.merge(pace_style, on=['Season', 'TeamID', 'Gender'], how='left')
merged_df = merged_df.merge(location_win, on=['Season', 'TeamID', 'Gender'], how='left')
merged_df = merged_df.merge(adjusted_efficiency, on=['Season', 'TeamID', 'Gender'], how='left')

# 3. 결측값 처리 (NaN -> 0으로 대체)
merged_df.fillna(0, inplace=True)

# 4. 최종 데이터 확인
print(merged_df.head())

# 5. 최종 데이터 저장
merged_df.to_csv('/kaggle/working/final_team_dataset.csv', index=False)

merged_df.head()

## # coach & massey 
- 남성데이터만 존재
- 칼럼 통일 후 상관관계 확인
- 상관계수가 높다면 활용

In [ ]:
# Massey Ordinals 데이터 전처리 필요
massey_ordinals = pd.read_csv('/kaggle/input/march-machine-learning-mania-2025/MMasseyOrdinals.csv')

# 2024년 시즌 데이터 필터링
season_2024 = massey_ordinals[massey_ordinals['Season'] == 2024]

# 각 TeamID별로 가장 최근 RankingDayNum 찾기
latest_team_rankings = season_2024.groupby(['TeamID', 'SystemName'])['RankingDayNum'].max().reset_index()

# 가장 최신 RankingDayNum을 가진 Massey Rating 가져오기
latest_massey_data = season_2024.merge(
    latest_team_rankings, on=['TeamID', 'SystemName', 'RankingDayNum'], how='inner'
)

# SystemName이 여러 개일 경우, 특정 시스템 선택 (예: 가장 신뢰할 수 있는 시스템 1개만 남김)
# SystemName 기준으로 신뢰도가 높은 시스템을 우선 선택 (예: "POM" or "MAS" 같은 신뢰할 수 있는 지수)
preferred_systems = ["POM", "MAS", "SAG"]  # 원하는 순위 시스템 리스트

latest_massey_data = latest_massey_data[latest_massey_data['SystemName'].isin(preferred_systems)]

# TeamID별 평균 Massey Rating 계산 (하나의 값으로 통합)
massey_ratings = latest_massey_data.groupby('TeamID')['OrdinalRank'].mean().reset_index()
massey_ratings.rename(columns={'OrdinalRank': 'MasseyRating'}, inplace=True)

# Season 및 Gender 추가 (남성 데이터만 있다고 가정)
massey_ratings['Season'] = 2024
massey_ratings['Gender'] = 'M'  # 현재 MMasseyOrdinals 데이터는 남성팀 데이터

# 최종 데이터 저장
massey_ratings.to_csv('/kaggle/working/processed_massey_ratings.csv', index=False)

massey_ratings.head()

In [ ]:
coach_stats = pd.read_csv('/kaggle/working/coach_stats.csv')
massey_ratings = pd.read_csv('/kaggle/working/processed_massey_ratings.csv')

# 기존 merged_df와 병합 전, 데이터의 공통 키 확인
print("merged_df columns:", merged_df.columns)
print("coach_stats columns:", coach_stats.columns)
print("massey_ratings columns:", massey_ratings.columns)

# 병합 전에 필요 없는 컬럼 제거 후, 병합
coach_stats = coach_stats[['Season', 'TeamID', 'Gender', 'CoachRating']]
massey_ratings = massey_ratings[['Season', 'TeamID', 'Gender', 'MasseyRating']]

# 기존 데이터와 병합 (일단 임시 병합하여 상관관계 확인)
temp_df = merged_df.merge(coach_stats, on=['Season', 'TeamID', 'Gender'], how='left')
temp_df = temp_df.merge(massey_ratings, on=['Season', 'TeamID', 'Gender'], how='left')

# 상관계수 계산
correlation_matrix = temp_df.corr()

# 상관관계 출력 (팀 승률, 토너먼트 성적과의 연관성 체크)
print(correlation_matrix[['RecentWinPct', 'TournamentAppearances', 'BestTournamentFinish']].loc[['CoachRating', 'MasseyRating']])

## 상관계수
- 1.00에 가까울수록 강한 양의 상관관계
- -1.00에 가까울수록 강한 음의 상관관계
- 0에 가까울수록 상관관계 없음

# 3. 전체 데이터 결합 및 시각화 : 상관관계 확인

## #1. 주요 변수 간의 상관 관계
- 중요 변수를 찾고, 불필요한 변수 제거 가능

In [ ]:
plt.figure(figsize=(12, 8))
sns.heatmap(merged_df.corr(), annot=True, cmap="coolwarm", fmt=".2f", linewidths=0.5)
plt.title("Feature Correlation Heatmap")
plt.show()

## #2. 공격 효율성(AdjOffEff)과 토너먼트 성적
- 공격력이 강한 팀일수록 NCAA토너먼트에서 높은 성적을 기록하는지 확인
- 공격 효율성이 성적과 상관이 높다면 중요 변수로 사용

In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(data=merged_df, x='AdjOffEff', y='MaxTournamentRound', hue='Gender', alpha=0.7)
plt.title("Adjusted Offensive Efficiency vs Tournament Performance")
plt.xlabel("Adjusted Offensive Efficiency")
plt.ylabel("Max Tournament Round")
plt.legend(title="Gender")
plt.show()

## #3. 홈-원정 경기 승률 차이와 토너먼트 성적
- 홈에서 강한 팀이 NCAA토너먼트에서도 높은 성적을 기록하는지 확인
- 홈 어드벤티지가 큰 팀을 구별할 수 있음

In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(data=merged_df, x='HomeWinPct', y='MaxTournamentRound', hue='Gender', alpha=0.7)
plt.title("Home Win % vs Tournament Performance")
plt.xlabel("Home Win Percentage")
plt.ylabel("Max Tournament Round")
plt.legend(title="Gender")
plt.show()

## #4. 컨퍼런스 강도와 토너먼트 성적
- 강한 컨퍼런스에 속한 팀이 NCAA토너먼트에서도 높은 성적을 기록하는지 확인
- 컨퍼런스 강도가 성적 예측에 중요한 정보인지 판단

In [ ]:
plt.figure(figsize=(10, 6))
sns.boxplot(data=merged_df, x='Gender', y='ConferencePowerIndex', hue='MaxTournamentRound')
plt.title("Conference Strength vs Tournament Performance")
plt.xlabel("Gender")
plt.ylabel("Conference Strength (CPI)")
plt.legend(title="Max Tournament Round")
plt.show()

## #5. 경기 스타일과 토너먼트 성적
- 빠른 템포(공격 속도)가 NCAA토너먼트에서도 높은 성적을 기록하는지 확인
- 경기 스타일이 토너먼트 성적에 영향을 주는지 평가 가능

In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(data=merged_df, x='Pace', y='MaxTournamentRound', hue='Gender', alpha=0.7)
plt.title("Game Tempo (Pace) vs Tournament Performance")
plt.xlabel("Pace (Possessions per Game)")
plt.ylabel("Max Tournament Round")
plt.legend(title="Gender")
plt.show()

## #6. 3점 슛 시도 비율과 토너먼트 성적
- 3점 슛을 많이 시도하는 팀이 NCAA토너먼트에서 유리한지 분석
- 3점 슛 중심 팀이 NCAA에서 좋은 성적을 거두는지 평가

In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(data=merged_df, x='ThreePtAttemptRatio', y='MaxTournamentRound', hue='Gender', alpha=0.7)
plt.title("Three-Point Attempt Ratio vs Tournament Performance")
plt.xlabel("Three-Point Attempt Ratio")
plt.ylabel("Max Tournament Round")
plt.legend(title="Gender")
plt.show()

## #7. 자유투 시도 비율과 토너먼트 성적
- 자유투 시도 비율이 높은 팀이 NCAA토너먼트에서도 좋은 성적을 내는지 분석

In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(data=merged_df, x='FreeThrowRate', y='MaxTournamentRound', hue='Gender', alpha=0.7)
plt.title("Free Throw Rate vs Tournament Performance")
plt.xlabel("Free Throw Rate")
plt.ylabel("Max Tournament Round")
plt.legend(title="Gender")
plt.show()

In [ ]:
from sklearn.feature_selection import SelectKBest, f_classif # Feature selection : 불필요한 변수 거거
from sklearn.decomposition import PCA # Principal Component Analysis(주성분 분석) : 모든 변수를 압축하여 적은 차원의 주요 정보만 유지

In [ ]:
all_adjusted_team_efficiency.head()

In [ ]:
# 전처리된 데이터 로드
efficiency = pd.read_csv('/kaggle/working/all_team_efficiency.csv')
win_pct = pd.read_csv('/kaggle/working/all_win_pct.csv')
tournament_stats = pd.read_csv('/kaggle/working/all_tournament_stats.csv')
conference_strength = pd.read_csv('/kaggle/working/all_conference_strength.csv')
pace_and_style = pd.read_csv('/kaggle/working/all_pace_and_style.csv')
adjusted_efficiency = pd.read_csv('/kaggle/working/all_adjusted_efficiency.csv')
location_win_pct = pd.read_csv('/kaggle/working/all_location_win_pct.csv')
coach_stats = pd.read_csv('/kaggle/working/coach_stats.csv')  # 남성팀만 사용 가능
massey_ordinals = pd.read_csv('/kaggle/working/massey_ordinals.csv')  # 남성팀만 사용 가능

# 토너먼트 성적 데이터 로드
tourney_results = pd.read_csv('/kaggle/input/march-machine-learning-mania-2025/MNCAATourneyCompactResults.csv')
tourney_results = tourney_results[['Season', 'WTeamID', 'LTeamID', 'DayNum']]  # 핵심 변수만 사용
tourney_results['Win'] = 1  # 승리 여부 추가

# 승리한 팀과 패배한 팀 데이터 변환
winners = tourney_results[['Season', 'WTeamID', 'Win']].rename(columns={'WTeamID': 'TeamID'})
losers = tourney_results[['Season', 'LTeamID']].rename(columns={'LTeamID': 'TeamID'})
losers['Win'] = 0

# 모든 팀의 토너먼트 성적 결합
tourney_team_results = pd.concat([winners, losers], ignore_index=True)

# 토너먼트 성적과 모든 전처리 데이터 병합
final_data = tourney_team_results \
    .merge(efficiency, on=['Season', 'TeamID'], how='left') \
    .merge(win_pct, on=['Season', 'TeamID'], how='left') \
    .merge(tournament_stats, on=['Season', 'TeamID'], how='left') \
    .merge(conference_strength, on=['Season', 'TeamID'], how='left') \
    .merge(pace_and_style, on=['Season', 'TeamID'], how='left') \
    .merge(adjusted_efficiency, on=['Season', 'TeamID'], how='left') \
    .merge(location_win_pct, on=['Season', 'TeamID'], how='left')

# 남성팀만 추가할 수 있는 데이터 (코치 정보, Massey Ordinals)
final_data = final_data.merge(coach_stats, on=['Season', 'TeamID'], how='left') \
    .merge(massey_ordinals, on=['Season', 'TeamID'], how='left')

# 최종 데이터 저장
final_data.to_csv('/kaggle/working/final_dataset.csv', index=False)

final_data.head()

# 3. 토너먼트 성적과 변수 간 상관관계 분석
- win(승리 여부)와 다른 변수와의 상관관계를 시각화하면 어떤 변수를 유지하고 제거할지 볼 수 있을 듯

In [ ]:
# 분석에 불필요한 변수 제외 (ID, Season, Gender 등)
corr_matrix = final_data.drop(['TeamID', 'Season', 'Gender'], axis=1).corr()

# 히트맵 시각화
plt.figure(figsize=(12, 8))
sns.heatmap(corr_matrix, cmap='coolwarm', linewidths=0.5, annot=False)
plt.title("Feature Correlation Heatmap")
plt.show()

## #. Massey Ordinals : 사용 X
- 작년 기준 마감일 이후 발표된 것으로 확인됨

In [ ]:
# Massey Ordinals 데이터
massey_ordinals = pd.read_csv('/kaggle/input/march-machine-learning-mania-2025/MMasseyOrdinals.csv')

# 2024년 시즌 데이터 필터링
season_2024 = massey_ordinals[massey_ordinals['Season'] == 2024]

# 각 SystemName별로 가장 최근 RankingDayNum 확인
latest_rankings = season_2024.groupby('SystemName')['RankingDayNum'].max().reset_index()

# RankingDayNum을 날짜로 변환 (시즌 시작일로부터의 일수로 가정)
# 예를 들어, 시즌 시작일이 2024년 11월 10일이라면:
season_start_date = pd.to_datetime('2024-11-10')

# RankingDayNum을 날짜로 변환
latest_rankings['RankingDate'] = season_start_date + pd.to_timedelta(latest_rankings['RankingDayNum'], unit='D')

# 결과 확인
print(latest_rankings)

In [ ]:
# WinLabel: 1이면 첫 번째 팀이 승리, 0이면 패배. (여기서는 간단한 규칙과 노이즈를 더해 생성)
df['WinLabel'] = ((df['538_pred'] + df['recent_win_pct'] + np.random.rand(n_samples)) > 1.5).astype(int)

# 사용할 피처와 타겟 설정
features = ['seed_diff', '538_pred', 'massey_ordinal', 'recent_win_pct', 'off_eff', 'def_eff']
target = 'WinLabel'

X = df[features]
y = df[target]

# 학습/테스트 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# 2. 기본 모델 정의
# RandomForest, XGBoost, LightGBM 모델을 개별 베이스 모델로
rf = RandomForestClassifier(n_estimators=100, random_state=42)
xgb_model = xgb.XGBClassifier(n_estimators=100, use_label_encoder=False, eval_metric='logloss', random_state=42)
lgb_model = lgb.LGBMClassifier(n_estimators=100, random_state=42)

# 각 모델을 튜플 형태로 리스트에 담기
estimators = [
    ('rf', rf),
    ('xgb', xgb_model),
    ('lgb', lgb_model)
]


In [ ]:
# 3. 스태킹 앙상블 모델 정의

In [ ]:
# 스태킹 앙상블 모델 학습
stack_clf.fit(X_train, y_train)

# 테스트 데이터에 대한 예측 확률
y_pred_proba = stack_clf.predict_proba(X_test)[:, 1]

# Log Loss 평가 (낮을수록 좋음)
ensemble_logloss = log_loss(y_test, y_pred_proba)
print(f"Ensemble Log Loss: {ensemble_logloss:.4f}")